In [1]:
import datetime
purchased_item =[] # A list about purchased drinks for only one transaction.
coin=[] #  A list that stores sales data for eachl buys  of one transaction.
moneyrecord ={}# keys area date and time, values are transaction amount at that time
purchaserecord ={}
stockrecord ={}

In [2]:
import pandas as pd
def owner_record():
    recordFile = pd.DataFrame(moneyrecord.items(), columns=['Date_Time', 'Sales_history ($)'])
    recordFile['Purchased_List'] = recordFile['Date_Time'].map(purchaserecord)
    recordFile['Stock_of_items'] = recordFile['Date_Time'].map(stockrecord)
    recordFile.to_csv("records", sep='\t', encoding='utf-8')
    

In [3]:
def record():
    
    with open('accounts.txt','w') as fileAccounts:
        line=0
        while line<=len(moneyrecord):
            line+=1
            for k,v in moneyrecord.items():
                tolog = f'\n Transaction time: {k}, transaction amount was: {v} '
                fileAccounts.write(tolog)
        
    with open('stock.txt','w') as fileStock:
        iline=0
        while iline<=len(stockrecord):
            iline+=1
            for k,v in stockrecord.items():
                tolog = f'\n Transaction time: {k}, stock of all items were: {v} '
                fileStock.write(tolog)
    
    with open('purchase.txt','w') as filePurchase:
        tline=0
        while tline<=len(purchaserecord):
            tline+=1
            for k,v in purchaserecord.items():
                tolog = f'\n Transaction time: {k}, the purchased items were: {v}'
                filePurchase.write(tolog)

In [4]:
# Initial Setting of the machine from the inventory data provided by the machine owner (customer).

def setmachine(stockormenu):
    '''
setmenu function takes one argument that can have any of the
two values:'stock' or 'menu'. Input of the data is taken from 
machine owner by a text file.
    '''
    file = open("initial_inventory.txt", "r")
    keys =[]
    price =[]
    stock =[]
    for aline in file:
        values = aline.split()
        keys.append(values[0])
        price.append(float(values[1]))
        stock.append(float(values[2]))
    file.close()
    if stockormenu=='stock':
        return dict(zip(keys,stock))
    elif stockormenu=='menu':
        return dict(zip(keys,price))
    else:
        print('Not valid input for setting the machine. Select stock or menu')
        

In [5]:
mainmenu = setmachine('menu')
main_stock = setmachine('stock')
print(mainmenu)

{'coke': 2.0, 'juice': 2.0, 'tea': 2.5, 'coffee': 2.5, 'fanta': 2.0, 'milk': 0.5, 'sugar': 0.5}


In [6]:
"""
class item:
    def __init__(self,drink,price,stock):
        self.drink = drink
        self.price = price
        self.stock = stock

coffee = item('Coffee',2,10)
tea = item('Tea',2,10)
juice = item('Juice',3,5)
coke = item('Coke', 1,0)
sugar=item('Sugar', 0.5,50)
milk = item('Milk', 0.5,20)

menu = [coffee, tea, juice, coke, sugar, milk]
"""

"\nclass item:\n    def __init__(self,drink,price,stock):\n        self.drink = drink\n        self.price = price\n        self.stock = stock\n\ncoffee = item('Coffee',2,10)\ntea = item('Tea',2,10)\njuice = item('Juice',3,5)\ncoke = item('Coke', 1,0)\nsugar=item('Sugar', 0.5,50)\nmilk = item('Milk', 0.5,20)\n\nmenu = [coffee, tea, juice, coke, sugar, milk]\n"

In [7]:
def displaymenu():
    for item in mainmenu:
        if main_stock[item] ==0:
            print(f'Sorry, no {item} available today !')
        else:
            if item == 'sugar' or item == 'milk':
                print(f'{item} price is $ {mainmenu[item]} extra if you choose for tea or coffee')
            else:
                print(f'{item} price is  $ {mainmenu[item]}')
    

In [8]:
def totalcost():
    cost =0
    for i in purchased_item:
        cost = cost+mainmenu[i]
    return cost

In [9]:
def totalsales():
    sales = 0
    for s in moneyrecord:
        sales+=s
    return sales

In [10]:
def deposit():
    coins=0
    for i in range(len(coin)):
        coins=coins+coin[i]
    return coins

In [11]:
def lowbalance(items):
    transact= input('You are in low balance. Do you wanna buy? Then press "y".'+'\n'+
                    'To change your order press "ch" '+'\n'+
                    'To cancel transaction or for no response, press any key except y & ch         ').lower()
    if transact == 'y':                                  
        morecoins = float(input(f'You need {totalcost()-deposit()+mainmenu[items]} dollars more to buy {items}.'
                                +'\n'+ 'Please enter more coins:                                 '))/100
        coin_validity(morecoins)
        drinks_func(items)
    elif transact=='ch':
        another_drink()
    else:
        change_func()   

In [12]:
def milkandsugar():
    print('wait please. Water is boiling'+'\n'+'Would you like to add milk and sugar?')
    if str(input('Do you need milk? Press "y" or "n"                                               ').lower())=='y':
        if deposit()-totalcost()>= mainmenu['milk']:
            print('ok, wait for a second!It will not take long. We are adding milk now.')
            main_stock['milk'] = main_stock['milk']-1 
            purchased_item.append('milk')
        else:
            lowbalance('milk')
            purchased_item.append('milk')
            print('ok, wait for a second! we are adding milk.')             
    else:
        print('ok, no milk.')
    sugar_add =str(input('Do you need to add sugar automatically? Press "y" or "n" or "manual"           ').lower())
    if sugar_add =='y' or sugar_add=='manual':
        if deposit()-totalcost()>= mainmenu['sugar']:
            if sugar_add =='y':
                print('ok, wait for a second!It will not take long. we are adding sugar automatically.')
            else:
                print('Your drink is stored. Collect sugar. Then add manually')
            main_stock['sugar'] = main_stock['sugar']-1
            purchased_item.append('sugar')
        else:
            lowbalance('sugar')
            if sugar_add =='y':
                print('ok, wait for a second!It will not take long. we are adding sugar automatically.')
            else:
                print('Your drink is stored. Collect sugar. Then add manually')       
    else:
        print('ok, no sugar.')

In [13]:
def change_func():  
    change = deposit()-totalcost()
    if purchased_item ==[]:
        print(f"You paid {deposit()} dollars and you bought nothing" +'\n'+
              f'Thank you. Hope to buy next time. Your change is {change} dollars ')
        print("Take your change please")
        print("Thank you.")
    else:
        moneyrecord[datetime.datetime.now().strftime("%d/%m/%Y--%H:%M:%S")] = totalcost()
        purchaserecord[datetime.datetime.now().strftime("%d/%m/%Y--%H:%M:%S")]=purchased_item
        stockrecord[datetime.datetime.now().strftime("%d/%m/%Y--%H:%M:%S")]= main_stock
        print(f"You paid {deposit()} dollars and you bought this drinks:{purchased_item}" +'\n'+
          f"Your total purchased amount is {totalcost()} dollars. So Your change is: {change} dollars")
        print("Take your change please")
        print("Thank you for buyeing. Goodbye")
        owner_record()
        restart()

In [14]:
def another_drink():
    j= str(input('Do you want another drink: y or n?                                              ')).lower()
    if j=='y':
        print(drink_selection())  
    else:
        print(change_func())
    

In [15]:
def no_stock():
    print('We are sorry, the item is not in stock; do you need another drink?')
    item= str(input('Do not want another drink? please select "n"'+'\n'+
                    'Or press any key to continue :                                              ')).lower()
    if item =='n':
        change_func()
    else: 
        #item=newitem
        drink_selection()
            

In [16]:
def not_in_menu(item):
    if item not in mainmenu:
            print('Sorry, your selection is not in the list.'+'\n'+
                  'Please select only listed drinks from the menu.')
            x= drink_selection()
    else:
            x= change_func()
    return x

In [17]:
def coin_validity(penny):
    coinlist = [.1,.2,.5,1,2]
    if penny not in coinlist:
        print('Sorry, we can take only specific coins.')
        penny = (float(input('Please enter your coins. Allowable coins are:'+'\n'+
                            '10cents,20cents,50cents,100cents,200cents                             '))/100)
    coin.append(penny)


In [18]:
def drink_confirmation(item):
    if item in mainmenu and item!='c':
        print(f'Cost of your selected drink: {item} is {mainmenu[item]}') #U-003Cost of the selected item is displayed
        
        confirm = str(input('Do you wanna proceed with your buys? then press "y" '+'\n'+ 
                        'Otherwise press "ch" for a change' +
                        '\n'+'or press "c" to cancel                                              ')).lower()
        
        if confirm == 'y': #U-004 Coin must be inserted to get the product
            initialcoins = (float(input('Please enter your coins. Allowable coins are:'+'\n'+
                                    '10cents,20cents,50cents,100cents,200cents                      '))/100)
            coin_validity(initialcoins)
            drinks_func(item)
        
        elif confirm == 'ch':
            print('Do you want to change purchasing? Then select new item ')
            item = str(input('Please select your drinks:                                            ')).lower()
            drink_confirmation(item)
        
        else: # U-005, The transaction shall be canceled if required
            change_func() 
    else:
        if item =='c': #item not in mainmenu
            change_func()   
        else:
            not_in_menu(item)

In [19]:
def drink_selection():
    displaymenu() # U-002, List of products is displayed
    item = str(input('Please select your drinks. For cancellation, press "c":                       ')).lower()
    print(f'Cost of your selected drink: {item} is ${mainmenu[item]}')
    drink_confirmation(item)      

In [20]:

def drinks_func(item):
        if main_stock[item]!=0:                           # when stock available
            if deposit()-totalcost()>= mainmenu[item]:
                
                main_stock[item] = main_stock[item]-1
                purchased_item.append(item)
                                                
                if item =='tea' or item =='coffee':       # choosing milk and sugar for tea and coffee.
                    print(milkandsugar())
                
                print(f'So far total costing is {totalcost()}'+'\n'+ 
                      f'So far, you purchased {purchased_item} drinks.'+'\n'
                      f'and your current balance is {deposit()-totalcost()} dollars')  
                
                another_drink()     # Choosing another drink 
            else:
                lowbalance(item)   # if user is on low-balance 
                              
        else:                     # zero stock condition
            no_stock()        
    
            

In [21]:
def restart():
    while True:
        answer=input("Please press 'y' or 'n' to commence a new transaction:              ")
        if answer not in ('y', 'n'):
            print("Invalid input. select 'y' or 'n'")
            restart()
        if answer == 'y':
            purchased_item.clear()
            coin.clear()
            print('Welcome to the vending machine.'+'\n'+ 
                  'This machine accepts coins from 10 cents to 2 dollars only.')
        else:
            print("Goodbye")
            break
    showmenu()
    

In [22]:
def showmenu():
    menuwant=input('Do you want to see menu now, then press "y"?'+'\n'+
                   'Or want to cancel? press any key                                       ')
    if menuwant == 'y':
            print('Here is the menu for your drinks ')
            displaymenu()
    else:
        print("Goodbye")
        

In [23]:
def drinkselection():
    displaymenu()
    item = str(input("Please select your drinks. For cancellation, press 'cancel':                       ")).lower()
    if item!='cancel':
        print(f'Cost of your selected drink: {item} is ${mainmenu[item]}')#U-003Cost of the selected item is displayed
    else:
        change_func()
    return item

In [24]:
drinkselection()

coke price is  $ 2.0
juice price is  $ 2.0
tea price is  $ 2.5
coffee price is  $ 2.5
Sorry, no fanta available today !
milk price is $ 0.5 extra if you choose for tea or coffee
sugar price is $ 0.5 extra if you choose for tea or coffee
Please select your drinks. For cancellation, press 'cancel':                       coke
Cost of your selected drink: coke is $2.0


'coke'

In [25]:
def drinkconfirmation(item):
    if item in mainmenu and item!='cancel':                                    
        confirm = str(input('Do you wanna proceed with your buys? press "y"'+'\n'+ 
                        'Otherwise press "ch" for change' +
                        '\n'+'or press "c" to cancel                                              ')).lower()
        
        if confirm == 'y': #U-004 Coin must be inserted to get the product
            initialcoins = (float(input('Please enter your coins. Allowable coins are:'+'\n'+
                                    '10cents,20cents,50cents,100cents,200cents                      '))/100)
            coin_validity(initialcoins)
            drinks_func(item)
        
        elif confirm == 'ch':
            print('Do you want to change purchasing? Then select new item ')
            item = str(input('Please select your drinks:                                            ')).lower()
            drinkconfirmation(item)
        
        else: # U-005, The transaction shall be canceled if required
            change_func() 
    else:
        if item =='cancel': #item not in mainmenu
            change_func()   
        else:
            not_in_menu(item)

In [ ]:
drinkconfirmation('tea')

Do you wanna proceed with your buys? press "y"
Otherwise press "ch" for change
or press "c" to cancel                                              y
Please enter your coins. Allowable coins are:
10cents,20cents,50cents,100cents,200cents                      200
You are in low balance. Do you wanna buy? Then press "y".
To change your order press "ch" 
To cancel transaction or for no response, press any key except y & ch         y
You need 0.5 dollars more to buy tea.
Please enter more coins:                                 50
wait please. Water is boiling
Would you like to add milk and sugar?
Do you need milk? Press "y" or "n"                                               y
You are in low balance. Do you wanna buy? Then press "y".
To change your order press "ch" 
To cancel transaction or for no response, press any key except y & ch         y
You need 0.5 dollars more to buy milk.
Please enter more coins:                                 50
So far total costing is 3.0
So far, you purchase

# Customer Requirements

In [ ]:
def stock_alert():
    for stock in main_stock:
        if main_stock[stock]>0.2 * initial_stock[stock] :
            print(f'There are {main_stock[stock]} number of {stock} in the machine. So no need to refil now.')
            #print(f'Dear customer, your machine is out of {stock}. Please fill up quickly')
        elif main_stock[stock]==0:
            print(f'There are only {main_stock[stock]} number of {stock} in the machine. Please refil right now.')
        else: # main_stock[stock]<= 0.2 * initial_stock[stock]:
            print(f'There are {main_stock[stock]} number of {stock} in the machine. So you should think about refilling')
        

In [ ]:
def reset():
    response = input('Do you want a factory("f") re-set?'+'\n'+'Or partial("p") re-set?' 
                     +'\n'+'or stop the machine("s")' +'\n'+'continue running for any key                        ')
    if response == 'f':
        print('Attention, Machine is resetting to factory re-set.' +'\n'+
              'Please take the record of your business transactions. All data shall be erased')
        purchased_item.clear()
        coin.clear()
        mainmenu.clear()
        main_stock.clear()
    elif response == 'p':
        print('Attention, Machine is resetting.' +'\n'+
                  'Please take the record of your business transactions. Purchase and money data shall be resetted')
        purchased_item.clear()
        coin.clear()
    elif response == 's':
        print('Machine is out of service for maintenance')
    else:
        print('Ok. machine is currently active.')
    return response

In [ ]:
reset()

In [ ]:
print(vendor_reset())

In [ ]:
def machine_active:
    if vendor_reset()!='f' and vendor_reset() !='p' and vendor_reset() !='s':
        print('Vendor Machine is running properly')
    else:
        print('Sorry, the machine is not in service')
   
        
    

In [ ]:
def customer():
    print('Welcome to the vending machine.'+'\n'+ 
          'This machine accepts coins from 10 cents to 2 dollars only.')
    

In [ ]:
record()